##### Importing required libraries
* from .env loading OpenAI key and other environment variables
* Official blog: <a href="https://langchain-ai.github.io/langmem/">link</a>

In [1]:
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore     # for demo; use AsyncPostgresStore in prod


# ---- LangMem: tools + background manager + (optional) short-term summarization helpers
from langmem import (
    create_manage_memory_tool,       # Hot-path tool to create/update/delete memories
    create_search_memory_tool,       # (Optional) Hot-path search tool
)
from langmem.short_term import summarize_messages, RunningSummary  # optional

import os
from dotenv import load_dotenv
load_dotenv()

True

* Choose the embedding model which we want to use the conversational information

In [2]:
store = InMemoryStore(index={"embed": "openai:text-embedding-3-small", "dims": 1536})

* Once the instance killed we will lose this memory being created now.
* It keeps increasing as it's internally handled keeps increasing till the instance is terminated.
* <b>Langmem</b>
    * Purpose‐built memory framework for agents:
        * Extraction, consolidation, namespaces, types of memory (semantic/episodic/procedural).
        * Simplifies memory management tasks (insert/update/delete, background managers, tools).
* Please check if we can replicate it with <b>vector database</b>
    * Drawback is it doesn't provide memory logic
    * We have to again write custom logic on top of this results from vectordb

In [3]:
manage_memory_tool = create_manage_memory_tool(
    namespace=("memories", "{langgraph_user_id}", "collection"),
    # Limit what the LLM is allowed to do if you prefer:
    # actions_permitted=("create", "update", "delete"),
    store=store
)

* It allows to search on the vector store data which was created by <b>manage_memory_tool</b>

In [4]:
search_memory_tool = create_search_memory_tool(
    namespace=("memories", "{langgraph_user_id}", "collection"),
    store=store
)


In [5]:
profile = {
    "name": "Ram",
    "full_name": "Ram",
    "user_profile_background": "You are the ruler of the world.",
}

In [6]:
agent_system_prompt_memory = """
< Role >
You are AI executive chat assistant. You are a capable of answering anything at ease using your knowledge. You can use your knowledge and history of conversation to answer the user's questions in a professional manner.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:
1. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
2. search_memory - Search for any relevant information that may have been stored in memory
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [7]:
def create_prompt(state):
    return [
        {
            "role": "system",
            "content": agent_system_prompt_memory.format(
                instructions="Use these tools when appropriate to help manage Rams's tasks efficiently.",
                **profile
            )
        }
    ] + state['messages']

In [8]:
tools= [
    # memory_manger,
    manage_memory_tool,
    search_memory_tool
]


* React agent
    * Give tools available
    * Prompt
    * If required for the usecase it would use the tools for our usecase depending on the prompt and the input.
    * It does execute using the tool
    * Here it uses these tools for managing memory:
        * <b>manage_memory_tool</b>
        * <b>search_memory_tool</b>

In [9]:
response_agent = create_react_agent(
    "openai:gpt-4o-mini",
    tools=tools,
    prompt=create_prompt,
    # Use this to ensure the store is passed to the agent
    store=store
)

In [10]:
config = {"configurable": {"langgraph_user_id": "session-id-test"}}

* Using react agent

In [11]:
response = response_agent.invoke(
    {"messages": [{"role": "user", "content": "My name is Ram."}]},
    config=config
)

In [12]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

My name is Ram.
================================== Ai Message ==================================
Tool Calls:
  manage_memory (call_SSzEd4uZ17go5QqyNpDFYlcI)
 Call ID: call_SSzEd4uZ17go5QqyNpDFYlcI
  Args:
    content: User's name is Ram.
    action: create
================================= Tool Message =================================
Name: manage_memory

created memory b5bb66e7-5f5d-4170-ad43-848108254cd0
================================== Ai Message ==================================

Nice to officially meet you, Ram! How can I assist you today?


In [13]:
response = response_agent.invoke(
    {"messages": [{"role": "user", "content": "John is my friend."}]},
    config=config
)

In [14]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

John is my friend.
================================== Ai Message ==================================
Tool Calls:
  manage_memory (call_q8ybiEWc0wMt8ap44hdKZE96)
 Call ID: call_q8ybiEWc0wMt8ap44hdKZE96
  Args:
    content: John is a friend of Ram.
    action: create
================================= Tool Message =================================
Name: manage_memory

created memory cf24da75-9c22-4d66-8fd8-000db67cdb04
================================== Ai Message ==================================

I've noted that John is your friend. If there's anything specific you'd like to remember or share about your friendship with him, feel free to let me know!


* It's using the memory and returning the data from the prior conversations

In [15]:
response = response_agent.invoke(
    {"messages": [{"role": "user", "content": "Who is John?"}]},
    config=config
)

In [16]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Who is John?
================================== Ai Message ==================================
Tool Calls:
  search_memory (call_03T96RTXqURzHsJw3vYs5QGo)
 Call ID: call_03T96RTXqURzHsJw3vYs5QGo
  Args:
    query: John
================================= Tool Message =================================
Name: search_memory

[{"namespace":["memories","session-id-test","collection"],"key":"cf24da75-9c22-4d66-8fd8-000db67cdb04","value":{"content":"John is a friend of Ram."},"created_at":"2025-10-27T12:51:05.567081+00:00","updated_at":"2025-10-27T12:51:05.567094+00:00","score":0.314446028352704},{"namespace":["memories","session-id-test","collection"],"key":"b5bb66e7-5f5d-4170-ad43-848108254cd0","value":{"content":"User's name is Ram."},"created_at":"2025-10-27T12:50:52.353626+00:00","updated_at":"2025-10-27T12:50:52.353638+00:00","score":0.16457608233286636}]
================================== Ai Message =========

* Please feel free to further experiment enhance wrt your use case
* Use AsyncPostgresStore for prod
    * We can auto-scale.
    * Memory is safe even if instance is down.
    * API memory doesn't keep increasing forever.
    * <a href="https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.postgres.AsyncPostgresStore">AsyncPostgresStore</a>